# Noise reduction + downsample

This stage applies a noise reduction step + a downsample
to reduce the sample rate to 16000Hz to train the tts faster.

Based on https://github.com/facebookresearch/denoiser

In [ ]:
%%capture

!pip install -U denoiser

In [ ]:
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio
from google.colab import drive
from pathlib import Path
from tqdm.notebook import tqdm
import soundfile as sf

In [ ]:
drive.mount('/content/drive/')
songs_path = Path("/content/drive/MyDrive/Máster/DLASP/Final/songs_separated/htdemucs/")


Mounted at /content/drive/


In [ ]:
model = pretrained.().cuda()

Downloading: "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th" to /root/.cache/torch/hub/checkpoints/dns64-a7761ff99a7d5bb6.th


  0%|          | 0.00/128M [00:00<?, ?B/s]

In [ ]:
for folder in (pbar := tqdm(list(songs_path.iterdir()), leave=False)):
  output_file = folder / 'vocals_denoised.wav'
  if not output_file.exists():
    pbar.set_description(folder.name)

    wav, sr = torchaudio.load(folder / 'vocals.wav')
    with torch.no_grad():
      wav = convert_audio(wav.cuda(), sr, model.sample_rate, model.chin)
      denoised = model(wav[None])[0]

    data = denoised.data.cpu().numpy()[0]
    sf.write(output_file, data, model.sample_rate)

  0%|          | 0/117 [00:00<?, ?it/s]